In [1]:
using IntervalConstraintProgramming, ValidatedNumerics

In [2]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

code = quote 
    z1 = x ^ 2
    z2 = y ^ 2
    z3 = z1 + z2
    z4 = sqrt(z3)
    z5 = 3 - z4
    z6 = z5 ^ 2
    z7 = z ^ 2
    z8 = z6 + z7
    z8 = z8 ∩ _A_
    (z8,z6,z7) = plusRev(z8,z6,z7)
    (z7,z,_) = powerRev(z7,z,2)
    (z6,z5,_) = powerRev(z6,z5,2)
    (z5,_,z4) = minusRev(z5,3,z4)
    (z4,z3) = sqrtRev(z4,z3)
    (z3,z1,z2) = plusRev(z3,z1,z2)
    (z2,y,_) = powerRev(z2,y,2)
    (z1,x,_) = powerRev(z1,x,2)
    return (x,y,z)
end


Separator with variables x, y, z

In [8]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time inner, boundary = setinverse(solid_torus, Y, 0.125);

 11.525667 seconds (95.68 M allocations: 2.872 GB, 15.36% gc time)


In [9]:
length(inner)

14880

## 3D vis 

In [10]:
using GLVisualize, GeometryTypes, Colors

In [11]:
positions = Point3{Float32}[Point3(mid(x)) for x in inner]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in inner]
colors = [RGBA(1f0, 0f0, 0f0, 0.1f0) for xx in inner];

In [12]:

using GLVisualize, GeometryTypes, GLAbstraction, Colors

println("Visualization packages loaded.")


println("Set inversion finished")

cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

# function cube(X::IntervalBox, inner=true)
#     lo = [x.lo for x in X]  # what's the good way to do this?
#     hi = [x.hi for x in X]
#     #color = Float32((hi[3] + 2.) / 4)
#     #c = Float32(abs(hi[3]) / 2.)
#     #mycolor = RGBA(1f0, c, c/2, 0.2f0)
#     if inner
#         mycolor = RGBA(1f0, 0f0, 0f0, 0.1f0)
#     else
#         mycolor = RGBA(0f0, 1f0, 0f0, 0.1f0)
#     end
#     return (HyperRectangle{3, Float32}(Vec3f0(lo), Vec3f0(hi - lo)), mycolor)
# end

window = glscreen()

# create an array of differently colored boxes in the direction of the 3 axes
#cubes = map(cube, inner2)
#boundarycubes = [cube(x, false) for x in boundary]

# convert to an array of normal meshes
# note, that the constructor is a bit weird. GLNormalMesh takes a tuple of
# a geometry and a color. This means, the geometry will be converted to a GLNormalMesh
# and the color will be added afterwards, so the resulting type is a GLNormalColorMesh
#meshes = map(GLNormalMesh, cubes)
# merge them into one big mesh
# the resulting type is a GLNormalAttributeMesh, since we merged meshes with different
# attributes (colors). An array of the colors will be created and each vertex in the
# mesh will be asigned to one of the colors found there.
#colored_mesh = merge(meshes)
#view(visualize(colored_mesh), window)

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

view(vis)


renderloop(window)

Visualization packages loaded.
Set inversion finished
